In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf

from datasets import load_dataset
import matplotlib.pyplot as plt

dataset = load_dataset("mnist")
train_data = dataset["train"]


Found cached dataset mnist (C:/Users/HP/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332)


  0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
num_samples = 0          # shows a sample number 
for i in range(num_samples):
    image = train_data[i]["image"]
    label = train_data[i]["label"]
    
    plt.imshow(image, cmap="gray")
    plt.title(f"Label: {label}")
    plt.show()

In [46]:
# only processing traing data need to do to rest when testing
input_data = np.zeros((len(train_data), 28, 28, 1)) # creates empty array to soon store the normaizled data
what_num = [-1]*len(train_data)

for i in range(num_samples):
    image = train_data[i]["image"]
    what_num[i] = train_data[i]["label"]
    input_data[i] = np.expand_dims(image, axis=-1) / 255.0  # so between 0-1

In [53]:
input_data = input_data.reshape(-1, 28, 28, 1)
print("Input data shape:", input_data.shape)

Input data shape: (60000, 28, 28, 1)


In [52]:
model = tf.keras.models.Sequential()
#from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model.add()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_7468\2016535502.py", line 5, in <module>
    model.add()
  File "C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\trackable\base.py", line 204, in _method_wrapper
    result = method(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\HP\AppData\Local\Packages\Python